In [ ]:
# DO NOT RUN THIS CELL
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySparkShell').getOrCreate()
sc = spark.sparkContext
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

In [1]:
spark

In [51]:
import pyspark.sql.functions as F

In [47]:
charts = spark.read.parquet('files/charts.parquet')
# charts = charts.toDF('id', 'title', 'position', 'date', 'countryId', 'chartName', 'movement', 'streams')
charts.createOrReplaceTempView('charts')
chartsRDD = sc.textFile('files/charts.csv').map(lambda line: line.split(','))

list(enumerate(charts.columns))

[(0, 'id'),
 (1, 'title'),
 (2, 'position'),
 (3, 'date'),
 (4, 'countryId'),
 (5, 'chartName'),
 (6, 'movement'),
 (7, 'streams')]

## Query 1

In [68]:
spark.sql('''
    SELECT sum(streams) FROM charts
    WHERE chartName="top200" AND title="Shape of You"
''').show()

+------------+
|sum(streams)|
+------------+
|  2324245979|
+------------+



In [70]:
title = 1
chartName = 5
streams = 7

(
    chartsRDD
     .filter(lambda x: x[chartName] == 'top200' and x[title] == 'Shape of You')
     .map(lambda x: int(x[streams]))
     .reduce(lambda x, y: x + y)
)

2324245979

## Query 2

In [65]:
(
    charts
     .filter(F.col('position') == 1)
     .groupBy(['chartName', 'title'])
     .count()
     .withColumn('avgCount', F.col('count') / 69)
     .groupBy('chartName')
     .agg(F.max('avgCount'))
     .show()
)

+---------+------------------+
|chartName|     max(avgCount)|
+---------+------------------+
|   top200|  54.2463768115942|
|  viral50|24.985507246376812|
+---------+------------------+



In [64]:
spark.sql('''
    SELECT chartName, first(title) title, max(avgCount) maxAvgTime
    FROM (
        SELECT chartName, title, count(*)/69 avgCount
        FROM charts
        WHERE position=1
        GROUP BY chartName, title
    )
    GROUP BY chartName
''').show()

+---------+--------------------+------------------+
|chartName|               title|        maxAvgTime|
+---------+--------------------+------------------+
|   top200|Chantaje (feat. M...|  54.2463768115942|
|  viral50|           I See Red|24.985507246376812|
+---------+--------------------+------------------+



In [63]:
title = 1
position = 2
chartName = 5

(
    chartsRDD
     .filter(lambda x: x[position] == '1')
     .map(lambda x: ((x[chartName], x[title]), 1))
     .reduceByKey(lambda x, y: x + y)
     .map(lambda x: (x[0][0], (x[0][1], x[1]/69)))
     .reduceByKey(lambda x, y: x if x[1] > y[1] else y)
     .map(lambda x: (x[0], *x[1]))
     .collect()
)

[('viral50', 'Calma - Remix', 24.985507246376812),
 ('top200', 'Shape of You', 54.2463768115942)]

## Query 3